# Homework 3.

Salette Guadalupe Noemi Villalobos A01246619

Erick Hernández Silva A01750170

Israel Sánchez Miranda A01378705

Loading imports and necessary libraries.

In [4]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim

from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from torchvision.models import resnet18
from torch.utils.data import Dataset, DataLoader
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold
from ucimlrepo import fetch_ucirepo 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder


# Letter recognition with ML

## Dataset Overview
The dataset used in this experiment is the Letter Recognition dataset from the UCI Machine Learning Repository. It contains 20,000 instances of character image features, where each instance represents a black-and-white pixel display of one of the 26 capital letters in the English alphabet. The objective is to classify each instance accurately as a particular letter based on 16 primitive numerical attributes. Each letter is represented with slight distortions across 20 different fonts, and the features capture properties like position, size, edge counts, and pixel correlations.

In [3]:
  
# fetch dataset 
letter_recognition = fetch_ucirepo(id=59) 
  
# data (as pandas dataframes) 
X = letter_recognition.data.features 
y = letter_recognition.data.targets 
  
# metadata 
print(letter_recognition.metadata) 
  
# variable information 
print(letter_recognition.variables) 


{'uci_id': 59, 'name': 'Letter Recognition', 'repository_url': 'https://archive.ics.uci.edu/dataset/59/letter+recognition', 'data_url': 'https://archive.ics.uci.edu/static/public/59/data.csv', 'abstract': 'Database of character image features; try to identify the letter', 'area': 'Computer Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 20000, 'num_features': 16, 'feature_types': ['Integer'], 'demographics': [], 'target_col': ['lettr'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1991, 'last_updated': 'Thu Sep 28 2023', 'dataset_doi': '10.24432/C5ZP40', 'creators': ['David Slate'], 'intro_paper': None, 'additional_info': {'summary': 'The objective is to identify each of a large number of black-and-white rectangular pixel displays as one of the 26 capital letters in the English alphabet.  The character images were based on 20 different fonts and each letter within these 20 fonts wa

## Experimental Setup
**Data Preparation and Encoding**: The dataset was loaded and split into features X and target y. As the target variable (lettr) is categorical (representing letters A-Z), it was encoded to numerical values for compatibility with the machine learning models.

**Data Splitting**: To evaluate model performance, we applied a 70-30 train-test split. This way, 70% of the data was used to train the models, while the remaining 30% served as a hold-out set to evaluate the model’s generalizability after hyperparameter tuning.

**Cross-Validation**: The 10-fold cross-validation (CV) technique was used on the training set to get an unbiased estimate of each model's performance. In 10-fold CV, the training data is divided into 10 subsets, with each subset used once as a validation set, and the remaining 9 subsets used for training. This process helps mitigate overfitting and provides a robust assessment of the model’s performance.

In [7]:
# Assuming 'y' is a DataFrame with a single column, we select that column for dtype check
if y.iloc[:, 0].dtype == 'object' or y.iloc[:, 0].nunique() > 1:
    encoder = LabelEncoder()
    y = encoder.fit_transform(y.iloc[:, 0])
else:
    y = y.iloc[:, 0]


In [8]:
# Split the data into train (70%) and hold-out (30%) sets
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


The following three machine learning models were selected for this experiment:

* **Random Forest**: A flexible, powerful ensemble method based on decision trees. It was chosen for its robustness and ability to capture complex patterns in the data.
* **Support Vector Machine (SVM)**: Known for high performance in classification tasks, especially with complex decision boundaries.
* **K-Nearest Neighbors (KNN)**: A straightforward yet effective method, chosen to evaluate how a simple distance-based classifier performs with this dataset.

Each model was evaluated with 10-fold cross-validation to obtain an initial performance estimate.

In [9]:
# Define the models to train and evaluate with cross-validation
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Support Vector Machine": SVC(random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier()
}

# Perform 10-fold cross-validation
cv_results = {}
for model_name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy')
    cv_results[model_name] = scores.mean()
    print(f"{model_name} 10-fold CV accuracy: {scores.mean():.4f}")


Random Forest 10-fold CV accuracy: 0.9539
Support Vector Machine 10-fold CV accuracy: 0.9149
K-Nearest Neighbors 10-fold CV accuracy: 0.9419



## Hyperparameter Tuning with GridSearchCV
We used GridSearchCV to find the best hyperparameters for each model. GridSearchCV performs an exhaustive search over a specified parameter grid, allowing us to identify the optimal settings for each model.

The following hyperparameters were tuned for each model:

* **Random Forest**: n_estimators (number of trees) and max_depth (maximum depth of each tree).
* **SVM**: C (regularization parameter) and kernel (type of kernel function).
* **KNN**: n_neighbors (number of neighbors) and weights (weight function).

In [12]:
# Tune hyperparameters with GridSearchCV
# Hyperparameter grids for each model
param_grids = {
    "Random Forest": {
        "n_estimators": [50, 100, 150],
        "max_depth": [None, 10, 20]
    },
    "Support Vector Machine": {
        "C": [0.1, 1, 10],
        "kernel": ['linear', 'rbf']
    },
    "K-Nearest Neighbors": {
        "n_neighbors": [3, 5, 7],
        "weights": ['uniform', 'distance']
    }
}

# Dictionary to store the best models after tuning
best_models = {}

# Tuning each model using GridSearchCV with 10-fold cross-validation
for model_name, model in models.items():
    print(f"Tuning hyperparameters for {model_name}...")
    grid_search = GridSearchCV(model, param_grids[model_name], cv=10, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_models[model_name] = grid_search.best_estimator_
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"Best cross-validated accuracy for {model_name}: {grid_search.best_score_:.4f}\n")


Tuning hyperparameters for Random Forest...
Best parameters for Random Forest: {'max_depth': 20, 'n_estimators': 150}
Best cross-validated accuracy for Random Forest: 0.9554

Tuning hyperparameters for Support Vector Machine...
Best parameters for Support Vector Machine: {'C': 10, 'kernel': 'rbf'}
Best cross-validated accuracy for Support Vector Machine: 0.9574

Tuning hyperparameters for K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'n_neighbors': 3, 'weights': 'distance'}
Best cross-validated accuracy for K-Nearest Neighbors: 0.9510



## Evaluation on Hold-Out Set
After identifying the best hyperparameters, each tuned model was evaluated on the hold-out set (30% of the data that was not used in training). This allows us to see how each model generalizes to unseen data.

In [11]:
# Evaluate the best models on the hold-out set
for model_name, best_model in best_models.items():
    y_pred = best_model.predict(X_holdout)
    accuracy = accuracy_score(y_holdout, y_pred)
    print(f"{model_name} hold-out set accuracy: {accuracy:.4f}")
    print(classification_report(y_holdout, y_pred, target_names=encoder.classes_))


Random Forest hold-out set accuracy: 0.9635
              precision    recall  f1-score   support

           A       0.99      0.98      0.99       237
           B       0.89      0.97      0.93       230
           C       0.96      0.95      0.96       221
           D       0.92      0.98      0.95       242
           E       0.97      0.96      0.96       230
           F       0.96      0.95      0.95       232
           G       0.95      0.94      0.95       232
           H       0.92      0.89      0.91       220
           I       0.97      0.92      0.95       226
           J       0.96      0.96      0.96       224
           K       0.95      0.94      0.94       222
           L       0.99      0.97      0.98       228
           M       0.99      0.98      0.99       238
           N       0.99      0.94      0.97       235
           O       0.97      0.96      0.96       226
           P       0.96      0.97      0.97       241
           Q       0.95      0.97    

We tested three machine learning models—Random Forest, Support Vector Machine (SVM), and K-Nearest Neighbors (KNN)—to classify letters based on image features. SVM achieved the highest accuracy on the hold-out set at 96.73%, closely followed by Random Forest at 96.35% and KNN at 95.48%. Hyperparameter tuning through cross-validation significantly enhanced each model's performance, confirming the importance of optimization. Overall, SVM demonstrated the best accuracy and generalizability, while Random Forest offers a strong alternative with the added benefit of feature interpretability.

## Machine Learning training and hyper parameter tuning.

## Statistical-based filter selection and Principal Component Analysis.

## Digit Recognition using Machine Learning.

In this section we use a Machine Learning (ML) algorithm to recognize the digits of one member of the team. We created a dataset that consists in $10$ images each one is a page with $224$ digits each that go from $0$ to $9$. Then our dataset $D$ is compund of digits $d_i$ such that $d_i\in\{0,1,2,3,4,5,6,7,8,9\}$. All digits are handwritten by one member of the team so the algorithm may be able to recognize the handwritting of such member. 

To train our model on the dataset we need to extract the features of each digit. The features we are going to extract are: 
1. The Hu moments. This features are scale and rotation invariant but can be sensitive to noise in the image.
2. The Euler number. This feature will help to distinguish digit holes. 

### Functions and considerations.

Firstly, we need to define a function to segment the $224$ digits from the source image. For this we define a function `segment_digits` that receives the path of the image and the output size of the segmented digit (by default we define an output image of $28\times28$). We apply a threshold to the image using a binary Otsu algorithm, this due to the simple nature of the image where we have a white background (the page itself) and a black object (the digit). Then we use the `cv2.findCountours` function to detect and isolate the individual handwritten digits in the image. We feed to this function the previously thresholded image and ask the algorithm to return the outer contours (`RETR_EXTERNAL`) to avoid nested contours of each digit. We store the values of each corner using the `cv2.CHAIN_APPROX_SIMPLE` format to speed up the process without losing significant information about the shape.

We then iterate over the contours found and calculate its bounding box using the `cv2.boundingRect` function and extract the digit from the thresholded image. We resize this image to the `output_size` and append it to the array that stores this images along with the coordinates of its bounding box.

In [11]:
def segment_digits(image_path, output_size=(28, 28)):
  """
  Segments digits from a handwritten digit image.

  Parameters:
  - image_path: Path to the image file.
  - output_size: Tuple indicating the size to resize the digit images.

  Returns:
  - A list of tuples (digit_image, bounding_box)
  """

  # Load image in grayscale
  img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

  # Threshold the image
  _, thresh = cv2.threshold(img, 255, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
  # Find contours of the digits
  contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  digit_images = []
  for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    digit = thresh[y:y+h, x:x+w]

    # Resize to standard size
    digit_resized = cv2.resize(digit, output_size, interpolation=cv2.INTER_AREA)
    digit_images.append((digit_resized, (x, y, w, h)))
  return digit_images

Now we define the function `get_features` which extracts the Hu moments and Euler number of a given image. To get the Hu moments the function uses the `cv2.moments` function and then it uses the `cv2.HuMoments` function to finally flatten the result. We make a log scale transformation to reduce the dynamic range of the moments; in this way we improve numerical stability and enchance the discriminative power of the features. 

To calculate the Euler number we threshold the image using the `cv2.THRESH_BINARY` threshold. We then get this number calculating the connected components and substracting the number of holes each digit has (he substract one always to take into account the backgorund that is labeled with `0`).

Finally, we store both the Hu moments and the Euler number in a list.

In [12]:
def get_features(image):
    """
    Calculates Hu moments and Euler number for a given image.

    Parameters:
    - image: The input image.

    Returns:
    - features: A list containing Hu moments and Euler number.
    """

    # Calculate Hu Moments
    moments = cv2.moments(image)
    hu_moments = cv2.HuMoments(moments).flatten()

    # Log scale transformation
    hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)

    # Calculate Euler number
    _, binary_image = cv2.threshold(image, 0, 1, cv2.THRESH_BINARY)
    euler_number = cv2.connectedComponents(binary_image.astype(np.uint8))[0] - 1

    features = list(hu_moments) + [euler_number]
    return features

Another function is defined; the `prepare_dataset`function takes the images from a folder with the following structure: 
- `folder/`
  - `label_0.jpeg`
  - `label_1.jpeg`
  - `label_2.jpeg`
  - `...`
  - `label_n.jpeg

  The function gets the label for each image and then applies the `segment_digits` to extract the digits of each image. Then, for each digit it gets the Hu moments and Euler number using the `get_features` function to finally store all these information into a dataframe.

In [13]:
def prepare_dataset(image_folder):
    """
    Prepares the dataset from images in a folder.

    Parameters:
    - image_folder: Folder containing images (one image per class, class name must be the name of the image).

    Returns:
    - DataFrame containing features and labels.
    """

    data = []
    labels = []
    for filename in os.listdir(image_folder):
        if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
            digit_label = os.path.splitext(filename)[0]
            image_path = os.path.join(image_folder, filename)

            digit_images = segment_digits(image_path)
            for digit_image, _ in digit_images:
                features = get_features(digit_image)
                data.append(features)
                labels.append(digit_label)

    columns = [f'Hu{i+1}' for i in range(7)] + ['EulerNumber']
    df = pd.DataFrame(data, columns=columns)
    df['Digit'] = labels
    return df

In [ ]:
In this experiment, we tested three machine learning models—Random Forest, Support Vector Machine (SVM), and K-Nearest Neighbors (KNN)—to classify letters based on image features. SVM achieved the highest accuracy on the hold-out set at 96.73%, closely followed by Random Forest at 96.35% and KNN at 95.48%. Hyperparameter tuning through cross-validation significantly enhanced each model's performance, confirming the importance of optimization. Overall, SVM demonstrated the best accuracy and generalizability, making it the most suitable model for deployment, while Random Forest offers a strong alternative with the added benefit of feature interpretability.

We used a $k$-Nearest Neighbors ($k$ NN) algorithm to recognize the digits. We define as default $k=3$. In this function we use the `Digit` column as the target value and use the `KNeighborsClassifier` class from `sklearn` for the $k$ NN algorithm. We used a $k$-Fold cross-validation with $10$ folds using a shuffle in our dataset to better train the model and have a more general overview of how it performs. We print the cross-validation scores and the mean score for each fold. 

In [14]:
def train_knn_model(df, k=3):
    """
    Trains a k-NN classifier using 10-fold cross-validation.

    Parameters:
    - df: DataFrame containing features and labels.
    - k: Number of neighbors for k-NN.

    Returns:
    - Trained k-NN model.
    - Cross-validation scores.
    """

    X = df.drop('Digit', axis=1)
    y = df['Digit']

    knn = KNeighborsClassifier(n_neighbors=k)
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    cv_scores = cross_val_score(knn, X, y, cv=kf)

    knn.fit(X, y)
    print(f'Cross-validation scores: {cv_scores}')
    print(f'Mean CV score: {cv_scores.mean()}')
    return knn, cv_scores

To test our ML algorithm we use the `classify_id_image` function. This function receives the path to an image which is the one containing the ID of a student written by the same member of the team that wrote the digits originally in order for the ML algorithm to recognize them. It firstly segments each digit found on the image using the `segment_digits`function and then for each digit found it gets its features. The function then organizes each feature in a dataframe and makes a prediction with the mdoel using the `predict` function of the $k$ NN model. Finally we sort the predicted digits based on their $x$ coordinate to mantain order, finally, the function returns the digit predictions. 

In [15]:
def classify_id_image(image_path, knn_model):
    """
    Classifies digits from an image containing the student ID.

    Parameters:
    - image_path: Path to the student ID image.
    - knn_model: Trained k-NN model.

    Returns:
    - List of predicted digits in order.
    """

    digit_images = segment_digits(image_path)

    digit_positions = []
    digit_features = []
    for digit_image, bbox in digit_images:
        features = get_features(digit_image)
        digit_features.append(features)
        digit_positions.append(bbox)

    # Predict digits
    X_new = pd.DataFrame(digit_features, columns=[f'Hu{i+1}' for i in range(7)] + ['EulerNumber'])
    predicted_digits = knn_model.predict(X_new)
    
    # Sort digits based on x-coordinate to maintain order
    digits_with_positions = list(zip(predicted_digits, digit_positions))
    digits_with_positions.sort(key=lambda x: x[1][0])  # Sort by x-coordinate
    ordered_digits = [digit for digit, _ in digits_with_positions]
    return ordered_digits


Another way we calculate the performance of the model is using the Levenshtein distance. This is, calculate how many deletions, insertions and substitutions we need to apply over a string $s_j$ to match an initial string $s_i$. This helps us to appreciate how similar is the predicted ID by the model with the original one.

In [16]:
def levenshtein_distance(s1, s2):
    """
    Calculates the Levenshtein distance between two strings.
    """
    
    if len(s1) < len(s2):
      return levenshtein_distance(s2, s1)
    
    # Initialize previous row of distances
    previous_row = list(range(len(s2) + 1))
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            # Cost of deletions, insertions, and substitutions
            deletions = previous_row[j + 1] + 1
            insertions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(deletions, insertions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

### Testing the algorihtm.

We tested the algorithm using thedataset defined at the [start of the section](#digit-recognition-using-machine-learning) using the `prepare_dataset` function over a folder with the structure defined at [Functions and considerations](#functions-and-considerations).

In [17]:
# Prepare the dataset
dataset_folder = 'digits'
df = prepare_dataset(dataset_folder)
print('Dataset prepared.')
df

Dataset prepared.


,Hu1,Hu2,Hu3,Hu4,Hu5,Hu6,Hu7,EulerNumber,Digit
0,3.139321,7.866246,9.607722,9.994978,-10.0,-9.999999,-10.0,1,0
1,2.470154,5.771757,9.556362,9.572124,10.0,-9.999607,-10.0,1,0
2,2.531530,6.019593,9.456941,9.087261,10.0,9.996963,10.0,1,0
3,2.438113,5.863366,9.253229,9.288303,10.0,9.998040,10.0,1,0
4,2.492358,5.891919,9.609553,9.323692,-10.0,9.998321,10.0,1,0
...,...,...,...,...,...,...,...,...,...
2669,2.680575,6.608726,8.188343,9.077046,-10.0,-9.998576,10.0,1,9
2670,2.618261,6.389477,7.991319,8.819356,10.0,-9.998544,10.0,1,9
2671,2.733390,7.603683,8.433619,9.446449,-10.0,9.999985,10.0,1,9
2672,2.676523,6.789756,8.161121,8.907812,10.0,-9.999589,10.0,1,9


We train our $k$ NN model using $k=2$. The model has a score of approximately $0.5$ which is not the best (basically a coin flip) but for educational purposes it works fine.

In [18]:
# Train the k-NN model
k = 2
knn_model, _ = train_knn_model(df, k=k)
print('k-NN model trained.')

Cross-validation scores: [0.57462687 0.55223881 0.45522388 0.54477612 0.52059925 0.4906367
 0.50187266 0.52434457 0.59550562 0.53558052]
Mean CV score: 0.5295404997484487
k-NN model trained.


We then read the ID handwritten image to test the algorithm.

In [19]:
# Classify digits from the student ID image
id_image_path = 'test.jpeg'
predicted_id = classify_id_image(id_image_path, knn_model)
print(f'Predicted Student ID: {"".join(map(str, predicted_id))}, Real ID is 1378705')

Predicted Student ID: 1378730, Real ID is 1378705


As we can see the algorithm works fairly fine on predicting and extracting the ID from the image. We apply the Levenshtein distance function to see how different are both strings. We also apply a normalized error rate to have a better overview of the proportion of the error.

In [20]:
# Calculate Levenshtein distance
predicted_id_str = "".join(map(str, predicted_id))
actual_id_str = '1378705'
error = levenshtein_distance(predicted_id_str, actual_id_str)
print(f'Levenshtein Distance (Error): {error}')

# Normalize Error Rate
max_length = max(len(predicted_id_str), len(actual_id_str))
error_rate = error / max_length
print(f'Normalized Error Rate: {error_rate:.2f}')

Levenshtein Distance (Error): 2
Normalized Error Rate: 0.29


The Levenshtein distance is of $2$, which means that at least two deletions, insertions or substitutions need to be applied for the predicted ID to be the same as the original ID. In this case we need two substitutions,change $s_6=3$ to $s_6=0$ and $s_7=0$ to $s_7=5$. The same way, we have an error rate of $0.29$ which suggest that almost a $30\%$ of the predicted ID is wrong. A way we can improve this algorithm is to do an exploratory search of the $k$ NN hyper parameters (number of neighbors and similar configurations) to achieve an optimal solution. We can also use different threshold values when doing the digit segmentation to avoid extracting digits incorrectly and confuse the model with this input images. Finally, another thing we can do to improve the algorithm is to increase the number of digit samples to have a bigger dataset. For this we would also try to balance more each digit class because in some cases a digit can be written in more than one way. This was not taken into account when handwritting the dataset and may have result in an inter-class imbalance and thus, impacting greatly on the model performance.

## Digit Recognition using Deep Learning.

It is also possible to do digit recognition using Deep Learning models. In this section we use a ResNet CNN and train it over the same dataset as [past section](#digit-recognition-using-machine-learning) to predict a student ID.

First we make a custom class `DigitDataset` that extends the `torch.utils.data.Dataset` class from PyTorch. This makes a custom dataset class to ease the compatibility between the data and the model. When we `__init__` the class we define an image directory and iterate over it to extract the labels and digits of an image using the `segment_digits()` function defined before. After that we append all this information to the `data` array.

We redefine the `__getitem__` method to obtain the image from the `data` array, we convert the image to RGB format since we are treating with binary images and then return the image and its label.

In [21]:
class DigitDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.data = []
        self.labels = []

        # Load data and labels
        for filename in os.listdir(image_dir):
            if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
                label = int(os.path.splitext(filename)[0])
                image_path = os.path.join(image_dir, filename)
                digit_images = segment_digits(image_path)  # Assuming this function segments and returns (image, label)
                
                for digit_image, _ in digit_images:
                    self.data.append(digit_image)
                    self.labels.append(label)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.data[idx]
        
        # Ensure the image has 3 channels for ResNet (H, W) -> (H, W, 3)
        if image.ndim == 2:
            image = np.stack([image] * 3, axis=-1)  # Convert grayscale to RGB

        image = Image.fromarray(image)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

We define a `transforms` object to ensure each digit image has the correct dimensions (28x28 pixels), converting each image to a tensor and normalize it to a range the model will be trained on and to help the model recognize digits even with minor variations in digit position or orientation.

In [22]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

Now, we define the function `create_resnet_model` function to create a ResNet18 CNN with 10 classes as default (for digits $0$ to $9$) and add a final linear layer for classification.

In [23]:
def create_resnet_model(num_classes=10):
    model = resnet18(pretrained=True)  # Load ResNet18 with pretrained weights
    model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modify the final layer
    return model

We now prepare both the model and the dataset, we define our dataset with its data folder and the transform object. Then we define a `DataLoader` object that will load the data into the model with a batch size of $32$ and `shuffle=True` to better generalize the training. Then, we use the `create_resnet_model`function to define our model; we also define the loss criterion (in this case the cross entropy loss) and an optimizer (Adam optimizer) with a learning rate of $0.001$. We will train this model for $10$ epochs.

In [24]:
# Parameters
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# Dataset and DataLoader
train_dataset = DigitDataset('digits', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Model, Loss, Optimizer
model = create_resnet_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

c:\Users\Israel\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Israel\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Israel/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:04<00:00, 10.6MB/s]


Sometimes training is too hard for a single CPU to handle, thus, if available, it is recommended to use a GPU (`cuda`) device to train the model. The line `model.to(device)` helps us to load the model into the device we will train it on.

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Now we define a `train` function, we make a loop for the number of epochs, set the model into training mode and initialize the total loss. We train the model in batches of size $32$. First we get the inputs and its corresponding labels from the batch of the dataloader and move them to the same device we are training the model on. We make a zero-gradient forward pass of the model and compute the loss function of the outputs and its true labels. Then we make a backward pass and update the weights of the model using this criterion. Finally, we track the total loss for each epoch to obtain the average loss of the epoch. 

In [ ]:
# Define the training function
def train(model, dataloader, optimizer, criterion, device, num_epochs):
    model.to(device)
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        model.train()
        total_loss = 0

        # Wrap the dataloader in tqdm for a progress bar
        for batch in tqdm(dataloader, desc=f"Training Epoch {epoch + 1}", unit="batch"):
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()  # Clear gradients
            
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights
            
            total_loss += loss.item()  # Track total loss for epoch

        avg_loss = total_loss / len(dataloader)
        print(f"Average Loss for Epoch {epoch + 1}: {avg_loss:.4f}")
        
    print("Training complete.")

Now we train the model using the functions and parameters defined before.

In [30]:
train(model, train_loader, optimizer, criterion, device, 10)

Epoch 1/10


Training Epoch 1: 100%|██████████| 84/84 [08:08<00:00,  5.81s/batch]


Average Loss for Epoch 1: 0.4044
Epoch 2/10


Training Epoch 2: 100%|██████████| 84/84 [08:19<00:00,  5.94s/batch]


Average Loss for Epoch 2: 0.3457
Epoch 3/10


...

Average Loss for Epoch 8: 0.2591
Epoch 9/10


Training Epoch 9: 100%|██████████| 84/84 [08:00<00:00,  5.72s/batch]


Average Loss for Epoch 9: 0.2399
Epoch 10/10


Training Epoch 10: 100%|██████████| 84/84 [07:51<00:00,  5.61s/batch]

Average Loss for Epoch 10: 0.2405
Training complete.


As we can see the model performs fairly well at the last epoch, having a relatively low loss of $0.2405$. Nevertheless, the model needs to be tested since this results may be due to overfit. It is also recommended to split the test dataset into a test and validation set to have a better overview of the performance. Also, cross-validation is another good tool for understanding the overall performance of a model.

Finally, we define a function that will segment the digits of the test image and for each digit found in the image we will predict it using our trained model. We set the model into evaluation mode, load the test image and make a prediction with this trained model.

In [50]:
# Define the prediction function using your CNN model
def predict_digits(model, image_path, transform, device):
    # Ensure model is in evaluation mode
    model.eval()

    # Segment digits from the image
    segmented_digits = segment_digits(image_path)
    
    # Initialize list to hold predictions
    predictions = []
    
    # Loop over each segmented digit
    for digit_image, _ in segmented_digits:
        # Convert the segmented digit to a PIL image
        digit_pil = Image.fromarray(digit_image).convert('RGB')
        
        # Apply the same transformations as used during training
        digit_tensor = transform(digit_pil).unsqueeze(0)  # Add batch dimension
        
        # Move tensor to the same device as the model
        digit_tensor = digit_tensor.to(device)
        
        # Make prediction
        with torch.no_grad():
            outputs = model(digit_tensor)
            _, pred = torch.max(outputs, dim=1)
            predictions.append(str(pred.item()))  # Append the predicted digit as a string
    
    # Join predictions to form the student ID
    student_id = ''.join(predictions)
    return student_id

We make the same process as before, we load the image, predict the digits on it and use the Levenshtein distance to obtain the error of the prediciton.

In [51]:
test_image_path = 'test.jpeg'
predicted_digit = predict_digits(model, test_image_path, transform, device)
print(f"The predicted digit is: {predicted_digit}")

error = levenshtein_distance(predicted_digit, actual_id_str)
print(f'Levenshtein Distance (Error): {error}')

# Normalize Error Rate
max_length = max(len(predicted_digit), len(actual_id_str))
error_rate = error / max_length
print(f'Normalized Error Rate: {error_rate:.2f}')

The predicted digit is: 5707831
Levenshtein Distance (Error): 6
Normalized Error Rate: 0.86


The Levenshtein distance is of $6$, which means that at least six deletions, insertions or substitutions need to be applied for the predicted ID to be the same as the original. The same way, we have an error rate of $0.86$ which suggest that almost an $86\%$ of the predicted ID is wrong. This very bad results may be due to an overfitting in the model and the same problems we have with the ML model: low data representation, small dataset and the segmentation model needs fine tuning to correctly extract the numbers of each page. Additionally, we can opt to fine tune the parameters of the model like learning rate, chose a better loss function, add warm-up epochs, use a validation dataset, etc.